In [1]:
import requests
import math
import os
import csv
import numpy as np
import pandas as pd

## Functions

In [18]:
trip

{'link': 'https://www.blablacar.de/trip?source=CARPOOLING&id=2192932945-potsdam-dresden',
 'waypoints': [{'date_time': '2021-05-13T22:00:00',
   'place': {'city': 'Potsdam',
    'address': 'Potsdam, Potsdam',
    'latitude': 52.391558,
    'longitude': 13.066694,
    'country_code': 'DE'}},
  {'date_time': '2021-05-14T00:20:00',
   'place': {'city': 'Dresden',
    'address': 'Bahnhof Dresden-Neustadt, Dresden',
    'latitude': 51.065532,
    'longitude': 13.741029,
    'country_code': 'DE'}}],
 'price': {'amount': '8.00', 'currency': 'EUR'},
 'vehicle': {'make': 'CITROEN', 'model': 'BERLINGO'},
 'distance_in_meters': 200037,
 'duration_in_seconds': 8400}

In [19]:
trip = all_trips[0]['trips'][0]
trip

{'link': 'https://www.blablacar.de/trip?source=CARPOOLING&id=2192645585-schoenefeld-dresden',
 'waypoints': [{'date_time': '2021-05-12T16:00:00',
   'place': {'city': 'Schönefeld',
    'address': 'An den Gehren, Schönefeld',
    'latitude': 52.38621,
    'longitude': 13.489772,
    'country_code': 'DE'}},
  {'date_time': '2021-05-12T18:10:00',
   'place': {'city': 'Dresden',
    'address': 'Peschelstraße 38, 01139 Dresden',
    'latitude': 51.084584,
    'longitude': 13.693945,
    'country_code': 'DE'}}],
 'price': {'amount': '9.00', 'currency': 'EUR'},
 'vehicle': {'make': 'SKODA', 'model': 'FABIA'},
 'distance_in_meters': 175525,
 'duration_in_seconds': 7800}

In [20]:
all_trips[0]['trips'][0]

{'link': 'https://www.blablacar.de/trip?source=CARPOOLING&id=2192645585-schoenefeld-dresden',
 'waypoints': [{'date_time': '2021-05-12T16:00:00',
   'place': {'city': 'Schönefeld',
    'address': 'An den Gehren, Schönefeld',
    'latitude': 52.38621,
    'longitude': 13.489772,
    'country_code': 'DE'}},
  {'date_time': '2021-05-12T18:10:00',
   'place': {'city': 'Dresden',
    'address': 'Peschelstraße 38, 01139 Dresden',
    'latitude': 51.084584,
    'longitude': 13.693945,
    'country_code': 'DE'}}],
 'price': {'amount': '9.00', 'currency': 'EUR'},
 'vehicle': {'make': 'SKODA', 'model': 'FABIA'},
 'distance_in_meters': 175525,
 'duration_in_seconds': 7800}

In [21]:
trip['vehicle']['model']

'FABIA'

In [6]:
def getRow(trip):
    '''
    Flattens a BlaBlaCar trip object into the plain list we want to upload to CARTO
    '''
    return {
            'url'         : trip['link'], #fixed
            'from_lat'    : trip['waypoints'][0]['place']['latitude'], #fixed
            'from_lon'    : trip['waypoints'][0]['place']['longitude'], #fixed
            'to_lat'      : trip['waypoints'][1]['place']['latitude'], #fixed
            'to_lon'      : trip['waypoints'][1]['place']['longitude'], #fixed
            'car_maker'   : trip['vehicle']['make'] if 'vehicle' in trip else '', #fixed
            'car_model'   : trip['vehicle']['model'] if 'vehicle' in trip else '' , #fixed
            'dep_date'    : trip['waypoints'][0]['date_time'], #fixed
            'arive_date'  : trip['waypoints'][1]['date_time'], #fixed
            'distance_m'  : trip['distance_in_meters'], #fixed
            'duration_s'  : trip['duration_in_seconds'], #fixed
            'price'       : trip['price']['amount'], #fixed
            'currency'    : trip['price']['currency'], #fixed
            }

Comfort and # of seats are missing. There are null values in the vehicle columns!

In [24]:
def getTrips(city):  #Berlin-Dresden
    '''
    Iterates over the BlaBlaCar /trips/ endpoint to retrieve all the pages
    '''
    trips = []
    page = 1
    
    URL = "https://public-api.blablacar.com/api/v3/trips?from_coordinate=48.864716,2.349014&to_coordinate=45.75801,4.8000158&locale=fr-FR&currency=EUR&start_date_local=2020-11-01T00:00:00&key=xcP0TPgR57BBbCUxjCI8G9S36kRVVnaO"
    KEY = os.getenv('BLABLACAR_API')
    CUR = "EUR"
    HEADERS = {
        'Content-Type': "application/json",
        'Cache-Control': "no-cache"
    }
    QS_BASE = {"key": KEY,"cur": CUR, "limit": 100} 

    querystring = dict( { 'page' : page, "fn": city, }, **QS_BASE)
    response = requests.request("GET", URL, headers=HEADERS, params=querystring)
    rj = response.json()

    while 'trips' in rj:
        #print('  Got page {}'.format(page))
        trips.extend(rj['trips'])
        page = page + 1

        querystring = dict( { 'page' : page, "fn": city }, **QS_BASE)
        response = requests.request("GET", URL, headers=HEADERS, params=querystring)
        rj = response.json()
        
    return trips

## Process cities

Process a list of cities to get all the trips and generates as a result a CSV ready to be used at CARTO

In [25]:
cities = ['Berlin']

In [26]:
all_trips = []

for city in cities:
    print('Getting the trips from BlaBlaCar API from {}...'.format(city))
    trips = getTrips(city)        
    print('{} trips retreived'.format(len(trips)))        
    all_trips.append({ 'city':city,'trips': trips})
    print('')

Getting the trips from BlaBlaCar API from Berlin...
0 trips retreived



In [10]:
print(all_trips)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Once get all the trips, save them into a single CSV file

In [12]:
filename = 'blablatripsDE3.csv'
print('Generating {}...'.format(filename))
with open(filename, 'w', newline='') as csvfile:
    fieldnames = [
        'city', 'url', 'from_lat', 'from_lon', 'to_lat', 'to_lon', 'car_maker', 'car_model', 
        'dep_date', 'arive_date', 'distance_m', 'duration_s', 'price', 'currency'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for group in all_trips:
        city = group['city']
        trips = group['trips']
        try:
            for trip in trips:
                row = getRow(trip)
                writer.writerow(dict({'city':city},**row))
        except KeyError as e:
            print(e)
            print(trip)

print('Done!')

Generating blablatripsDE3.csv...
Done!


In [13]:
df = pd.read_csv('blablatripsDE3.csv')

In [14]:
df.shape

(11070, 14)

In [17]:
df.head(50)

,city,url,from_lat,from_lon,to_lat,to_lon,car_maker,car_model,dep_date,arive_date,distance_m,duration_s,price,currency
0,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.386210,13.489772,51.084584,13.693945,SKODA,FABIA,2021-05-12T16:00:00,2021-05-12T18:10:00,175525,7800,9.0,EUR
1,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.520812,13.699738,51.065486,13.740971,NaN,NaN,2021-05-12T16:30:00,2021-05-12T18:50:00,201180,8400,8.0,EUR
2,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.571338,13.565649,51.065486,13.740971,NaN,NaN,2021-05-12T17:00:00,2021-05-12T19:30:00,214728,9000,8.0,EUR
3,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.392216,13.790521,51.056018,13.708131,NaN,NaN,2021-05-12T18:00:00,2021-05-12T19:50:00,191765,6600,7.0,EUR
4,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.478608,13.320768,51.040502,13.731419,NaN,NaN,2021-05-12T21:20:00,2021-05-12T23:40:00,194244,8400,11.0,EUR
5,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.364441,13.509853,51.085411,13.692314,VOLKSWAGEN,PASSAT,2021-05-12T22:20:00,2021-05-13T00:00:00,167673,6000,8.0,EUR
6,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.466690,13.488654,51.084884,13.694924,NaN,NaN,2021-05-13T10:00:00,2021-05-13T11:50:00,178662,6600,7.0,EUR
7,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.451503,13.320846,51.056018,13.708131,NaN,NaN,2021-05-13T12:00:37,2021-05-13T14:00:37,192747,7200,7.0,EUR
8,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.507300,13.280596,51.040502,13.731419,FORD,FUSION,2021-05-13T16:10:00,2021-05-13T18:30:00,196844,8400,11.0,EUR
9,Berlin,https://www.blablacar.de/trip?source=CARPOOLIN...,52.435094,13.541023,51.042667,13.796564,FORD,FOCUS,2021-05-13T19:00:00,2021-05-13T21:00:00,180748,7200,10.0,EUR


In [27]:
df.info

<bound method DataFrame.info of          city                                                url   from_lat  \
0      Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.386210   
1      Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.520812   
2      Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.571338   
3      Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.392216   
4      Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.478608   
...       ...                                                ...        ...   
11065  Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.466690   
11066  Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.451503   
11067  Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.507300   
11068  Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.435094   
11069  Berlin  https://www.blablacar.de/trip?source=CARPOOLIN...  52.391558   

        from_lon   